# Lab 1: Set Covering

First lab + peer review. List this activity in your final report, it will be part of your exam.

## Task

Given a number $N$ and some lists of integers $P = (L_0, L_1, L_2, ..., L_n)$,
determine is possible $S = (L_{s_0}, L_{s_1}, L_{s_2}, ..., L_{s_n})$
such that each number between $0$ and $N-1$ appears in at least one list

$$\forall n \in [0, N-1] \ \exists i : n \in L_{s_i}$$

and that the total numbers of elements in all $L_{s_i}$ is minimum.

## Input generator

In [13]:
SEED = 42

def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

# Gready Solution (Professor's solution)

In [14]:
import random
import logging
import numpy as np

def greedy(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    while goal != covered:
        x = all_lists.pop(0)
        if not set(x) < covered:
            solution.append(x)
            covered |= set(x)
    logging.info(
        f"Greedy solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")


logging.getLogger().setLevel(logging.INFO)
for N in [5, 10, 20, 100, 500, 1000]:
     greedy(N)

INFO:root:Greedy solution for N=5: w=5 (bloat=0%)
INFO:root:Greedy solution for N=10: w=13 (bloat=30%)
INFO:root:Greedy solution for N=20: w=46 (bloat=130%)
INFO:root:Greedy solution for N=100: w=332 (bloat=232%)
INFO:root:Greedy solution for N=500: w=2162 (bloat=332%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)


# Breadth First

## Generative Search function

In [15]:
from gx_utils import PriorityQueue

In [16]:
class State:
    def __init__(self, data: np.ndarray):
        self._data = data.copy()
        self._data.flags.writeable = False

    def __hash__(self):
        #return hash(tuple(tuple(x) for x in self.copy_data().tolist()))
        return hash(bytes(self._data))

    def __eq__(self, other):
        return bytes(self._data) == bytes(other._data)

    def __lt__(self, other):
        return bytes(self._data) < bytes(other._data)

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

In [17]:
#return a new_state if elem contains element different from state
def result(state, elem):
    new_state = []
    [new_state.append(e) for e in state.copy_data().tolist()]
    new_state.append(elem)
    new_state = sorted(new_state, key=lambda l: len(l))

    return State(np.array(new_state))

def is_valid(state, elem):
    covered = set()
    for e in state.copy_data():
        [covered.add(i) for i in e]
    old_covered = covered.copy()
    covered |= set(elem)
    if old_covered == covered:
        return False
    else:
        return True


def goal_test(state, goal_arr):
    state_set = set()
    test = state.copy_data().tolist()
    for e in test:
        [state_set.add(i) for i in e]
    state_set = sorted(list(state_set))
    return state_set == goal_arr

def possible_actions(state: State):
    #return [item for item in universe if item not in state.copy_data().tolist()]
    return [tuple(item) for item in input_state if is_valid(state, item)]

In [18]:
from typing import Callable


def search(
    initial_state: State,
    goal_test: Callable,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
    N: int
):
    frontier = PriorityQueue()
    parent_state.clear()
    state_cost.clear()

    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0

    goal_array = list(range(N))
    nodes = 0
    while state is not None and not goal_test(state, goal_array):
        for a in possible_actions(state):

            if not is_valid(state, a):
                continue
            new_state = result(state, a)
            cost = unit_cost(a)

            if new_state not in state_cost and new_state not in frontier:
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                frontier.push(new_state, p=priority_function(new_state))
                logging.debug(f"Added new node to frontier (cost={state_cost[new_state]})")
            elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                old_cost = state_cost[new_state]
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                logging.debug(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
        if frontier:
            state = frontier.pop()
            nodes += 1
        else:
            state = None

    path = list()
    s = state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]

    logging.info(f"N = {N} | Found a solution in {len(path):,} steps; visited {len(state_cost):,} states, nodes: {nodes}, w = {sum(len(x) for x in path[0])}")
    logging.info(f'state: {path[0]}')
    return list(reversed(path))

## Breadth First with search function

In [19]:
parent_state = dict()
state_cost = dict()

logging.getLogger().setLevel(logging.INFO)
''',10,20,50,100,500,1000'''
final = []
for n in [5, 10, 20]:
    input_state = sorted(problem(n, seed=42), key=lambda l: len(l))
    input_state = sorted(set(tuple(sorted(x)) for x in input_state), key=lambda l: len(l))
    initial_state = State(np.array([input_state[0]]))

    logging.debug(f'initial_state: {initial_state}')
    logging.debug(f'initial nodes: {len(input_state)}')
    final = search(
        initial_state=initial_state,
        goal_test=goal_test,
        parent_state=parent_state,
        state_cost=state_cost,
        priority_function=lambda s: len(state_cost),
        unit_cost=lambda a: 1,
        N=n
    )

C:\Users\aless\AppData\Local\Temp\ipykernel_17488\3151961595.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return State(np.array(new_state))
INFO:root:N = 5 | Found a solution in 4 steps; visited 329 states, nodes: 79, w = 5
INFO:root:state: [list([2]) list([4]) list([0]) (1, 3)]
INFO:root:N = 10 | Found a solution in 4 steps; visited 50,250 states, nodes: 2372, w = 10
INFO:root:state: [list([6]) (0, 5) (2, 7, 8) (1, 3, 4, 9)]
INFO:root:N = 20 | Found a solution in 5 steps; visited 361,995 states, nodes: 26556, w = 27
INFO:root:state: [list([7, 17, 18]) list([5, 8, 16]) (4, 6, 15, 17, 18)
 (0, 1, 3, 7, 9, 10, 11, 15) (2, 8, 12, 13, 14, 16, 17, 19)]


In [ ]:
parent_state = dict()
state_cost = dict()

logging.getLogger().setLevel(logging.INFO)

#for n in [5,10,20,50,100,500,1000]:
for N in [5]:
    GOAL = set(range(N))
    COVERED = set()

    def goal_test(state):
        return GOAL != COVERED

    input_state = sorted(problem(N, seed=42), key=lambda l: len(l))
    initial_state = State(np.array([]))
    search(
        initial_state=initial_state,
        goal_test=goal_test,
        parent_state=parent_state,
        state_cost=state_cost,
        priority_function=lambda s: len(state_cost),
        unit_cost=lambda a: len(a)
    )